# Phase 5 Supplemental Notebook

Before beginning the modeling aspect of this project, I had to obtain the data for modeling as well as results. In this notebook I go through the process of accessing the NHL API, scraping web data, constructing and exporting the dataframes, and obtaining the same statistics for this season's upcoming free agents.

In [1]:
import requests
import json
import pandas as pd
import string
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns
from collections import Counter

## User-defined functions

I create user-defined functions to handle most of the raw data cleaning and compiling of data frames. The first function goes through the process of contacting a website, [CapFriendly](https://www.capfriendly.com), that offers free historical contract data. The second and third functions take the output of the first function and compile a dataframe with engineered features.

In [2]:
def getPlayerContractData(df_players,cap_table):
    """
    Returns a dictionary with key corresponding to player and value corresponding to dictionary of two tables:
    The first table is contract specific data and player-immutable data, and the second table is advanced player
    statistics broken down by year
    """
    
    dict_contracts = {}
    
    for p in list(df_players.index):
        first,last = p.split(' ')[0],p.split(' ')[1]
        url = f'https://www.capfriendly.com/players/{first}-{last}'
        response = requests.get(url)
        try:
            tables = pd.read_html(response.text)
        except ValueError:
            continue
        
        if len(tables) < 2:
            continue
        dict_player = {}
        df_allContracts = pd.DataFrame()
        for t in (tables[:-1]):
            t_new = t.iloc[1:,[0,2]]
            contract_length = t_new.shape[0]-1
            start_year = (int(t_new.iloc[0,0].split('-')[0]) if isinstance(t_new.iloc[0,0].split('-')[0],str) 
                          else None)
            total_value = (int(t_new.iloc[-1,1].replace('$','').replace(',','')) if isinstance(t_new.iloc[-1,1],str) 
                           else None)
            df_allContracts.loc[start_year,'Value'] = total_value
            df_allContracts.loc[start_year,'Length'] = contract_length

        df_allContracts = df_allContracts.dropna(axis=0)
        if (df_allContracts.shape[0] < 1):
            continue
        df_allContracts['AAV'] = (df_allContracts['Value']/df_allContracts['Length'])
        for y in list(df_allContracts.index):
            df_allContracts.loc[y,'Contract Age'] = y-df_players.loc[p,'birthYear']
            if y >= 2005:
                df_allContracts.loc[y,'Cap Hit'] = df_allContracts.loc[y,'AAV']/cap_table.loc[y,'UPPER LIMIT']
            else:
                df_allContracts.loc[y,'Cap Hit'] = 0
        df_allContracts = df_allContracts[df_allContracts['Cap Hit'] > 0]
        
        df_allContracts['Position'] = df_players.loc[p,'Position']
        df_allContracts['Nationality'] = df_players.loc[p,'nationality']
        df_allContracts['Height'] = df_players.loc[p,'height']
        df_allContracts['Overall Pick'] = df_players.loc[p,'Overall Pick']
        
        summary_table = tables[-1]
        summary_table = summary_table[~summary_table['SEASON'].isna()].iloc[:-1,:]
        summary_table = summary_table[summary_table['LEAGUE'] == 'NHL']
        summary_table['SEASON ENDING'] = (summary_table['SEASON'].apply(lambda x: int(x.split('-')[0][:2] + 
                                                                                      x.split('-')[1])))
        summary_table = summary_table.set_index('SEASON ENDING')

        dict_player['Summary'] = df_allContracts
        dict_player['Stats'] = summary_table
        dict_contracts[p] = dict_player
        
    return dict_contracts

In [3]:
def getDataTable_g(dict_players):
    """
    Returns a table with contract data that can be further processed for modeling use
    Specifically returns a table for modeling goalie contracts
    """
    
    df_final = pd.DataFrame()
    
    for p in list(dict_players.keys()):
        
        df_stat_summary = dict_players[p]['Stats'].drop(columns=['SEASON','TEAM','LEAGUE','Unnamed: 3','Unnamed: 7',
                                                                 'Unnamed: 15','PLAYOFFS.1','PLAYOFFS'])
        df_stat_summary.columns = ['GAMES_PLAYED','GAA','Sv%','GP_PLAYOFF','GAA_PLAYOFF','Sv%_PLAYOFF','GA60',
                                   'xGA60','GSAx60','GA60_PLAYOFF','xGA60_PLAYOFF','GSAx60_PLAYOFF']
        df_stat_summary = df_stat_summary.fillna(0)
        df_stat_summary = df_stat_summary.applymap(lambda x: pd.to_numeric(x) if x != '-' else 0)

        df_contract_allData = dict_players[p]['Summary'].copy()

        df_data = pd.DataFrame()

        for y in list(dict_players[p]['Summary'].index):
            ct_years = [y,y-1,y-2]
            ct_year_symbols = ['_t','_t1','_t2']
            ls_temp = []
            t_count = 0
            for t in ct_years:
                if t in df_stat_summary.index:
                    row_t = df_stat_summary.loc[t]
                    row_t.index = [c + ct_year_symbols[ct_years.index(t)] for c in row_t.index]
                    df_temp = pd.DataFrame(row_t).T
                    df_temp.index = [y]
                    ls_temp.append(df_temp)
                    t_count += 1
                else:
                    df_temp = pd.DataFrame(0,columns = [c + ct_year_symbols[ct_years.index(t)] for c in 
                                                        df_stat_summary.columns],index=[y])
                    ls_temp.append(df_temp)

            if t_count > 0:
                avg = (np.array(ls_temp[0].values)+np.array(ls_temp[1].values)+(ls_temp[2].values))/t_count
                avg_columns = [c + '_avg' for c in df_stat_summary.columns]
                avg_df = pd.DataFrame(avg,columns=avg_columns,index=[y])
                df_y = pd.concat([ls_temp[0],ls_temp[1],ls_temp[2],avg_df],axis=1)

                if df_data.empty:
                    df_data = df_y
                else:
                    df_data = pd.concat([df_data,df_y],axis=0)

            else:
                df_contract_allData = df_contract_allData.drop(labels=[y],axis=0)

        df_final_player = df_contract_allData.merge(df_data,left_index=True,right_index=True)
        
        if df_final.empty:
            df_final = df_final_player
        else:
            df_final = pd.concat([df_final,df_final_player],axis=0)
    
    return df_final

In [4]:
def getDataTable_ng(dict_contracts):
    """
    Returns a table with contract data that can be further processed for modeling use
    Specifically returns a table for modeling non-goalie contracts
    """
    
    df_final = pd.DataFrame()
    
    for p in list(dict_contracts.keys()):
        df_stat_summary = dict_contracts[p]['Stats'].drop(columns=['SEASON','TEAM','LEAGUE','Unnamed: 3',
                                                                   'Unnamed: 10','Unnamed: 25','Unnamed: 33',
                                                                   'PLAYOFFS','G.3','G.2'])
        df_stat_summary.columns = ['GAMES_PLAYED','G','A','P','PLUS_MINUS','PIM','GP_PLAYOFF','G_PLAYOFF','A_PLAYOFF',
                                   'P_PLAYOFF','PLUS_MINUS_PLAYOFF','PIM_PLAYOFF','TOI','ixG','xG±/60','RelxG±/60',
                                   'C±/60', 'RelC±/60','TOI_PLAYOFF','ixG_PLAYOFF','xG±/60_PLAYOFF',
                                   'RelxG±/60_PLAYOFF','C±/60_PLAYOFF','RelC±/60_PLAYOFF']
        df_stat_summary = df_stat_summary.fillna(0)
        df_stat_summary['TOI'] = df_stat_summary['TOI'].apply(lambda x: (x.split(':')))
        df_stat_summary['TOI'] = df_stat_summary['TOI'].apply(lambda x: int(x[0])+(pd.to_numeric(x[1])/60) 
                                                              if len(x) > 1 else 0)
        df_stat_summary['TOI_PLAYOFF'] = df_stat_summary['TOI_PLAYOFF'].apply(lambda x: (x.split(':')))
        df_stat_summary['TOI_PLAYOFF'] = df_stat_summary['TOI_PLAYOFF'].apply(lambda x: int(x[0]) + 
                                                                              (pd.to_numeric(x[1])/60) 
                                                                              if len(x) > 1 else 0)
        df_stat_summary = df_stat_summary.applymap(lambda x: pd.to_numeric(x) if x != '-' else 0)

        df_stat_summary['G_PG'] = df_stat_summary.apply(lambda row: (row.G/row.GAMES_PLAYED 
                                                                     if row.GAMES_PLAYED > 0 else 0),axis=1)
        df_stat_summary['P_PG'] = df_stat_summary.apply(lambda row: (row.P/row.GAMES_PLAYED 
                                                                     if row.GAMES_PLAYED > 0 else 0),axis=1)
        df_stat_summary['PIM_PG'] = df_stat_summary.apply(lambda row: (row.PIM/row.GAMES_PLAYED 
                                                                       if row.GAMES_PLAYED > 0 else 0),axis=1)

        df_stat_summary['G_PG_PLAYOFF'] = df_stat_summary.apply(lambda row: (row.G_PLAYOFF/row.GP_PLAYOFF 
                                                                             if row.GP_PLAYOFF > 0 else 0),axis=1)
        df_stat_summary['P_PG_PLAYOFF'] = df_stat_summary.apply(lambda row: (row.P_PLAYOFF/row.GP_PLAYOFF 
                                                                             if row.GP_PLAYOFF > 0 else 0),axis=1)
        df_stat_summary = df_stat_summary.applymap(lambda x: pd.to_numeric(x) if x != '-' else 0)
        
        df_contract_allData = dict_contracts[p]['Summary'].copy()

        df_data = pd.DataFrame()

        for y in list(dict_contracts[p]['Summary'].index):
            ct_years = [y,y-1,y-2]
            ct_year_symbols = ['_t','_t1','_t2']
            ls_temp = []
            t_count = 0
            for t in ct_years:
                if t in df_stat_summary.index:
                    row_t = df_stat_summary.loc[t]
                    row_t.index = [c + ct_year_symbols[ct_years.index(t)] for c in row_t.index]
                    df_temp = pd.DataFrame(row_t).T
                    df_temp.index = [y]
                    ls_temp.append(df_temp)
                    t_count += 1
                else:
                    df_temp = pd.DataFrame(0,columns = [c + ct_year_symbols[ct_years.index(t)] 
                                                        for c in df_stat_summary.columns],index=[y])
                    ls_temp.append(df_temp)

            if t_count > 0:
                avg = (np.array(ls_temp[0].values)+np.array(ls_temp[1].values)+(ls_temp[2].values))/t_count
                avg_columns = [c + '_avg' for c in df_stat_summary.columns]
                avg_df = pd.DataFrame(avg,columns=avg_columns,index=[y])
                df_y = pd.concat([ls_temp[0],ls_temp[1],ls_temp[2],avg_df],axis=1)

                if df_data.empty:
                    df_data = df_y
                else:
                    df_data = pd.concat([df_data,df_y],axis=0)

            else:
                df_contract_allData = df_contract_allData.drop(labels=[y],axis=0)

        df_final_player = df_contract_allData.merge(df_data,left_index=True,right_index=True)
        
        if df_final.empty:
            df_final = df_final_player
        else:
            df_final = pd.concat([df_final,df_final_player],axis=0)
    
    return df_final

## API Scraping

A lot of the player-specific (non-contract) data for this project comes from the NHL API. There is a useful [GitHub repository](https://github.com/dword4/nhlapi) that lists useful links for accessing the API. Since 2006, the NHL has used a hard salary cap limit to dictate newly signed player contracts. To obtain contract data, I looked at players who have been active since 2006 to balance getting as much historical data as possible while also staying within the context of how teams and contracts are currently constructed. To do this, I went through the rosters of current NHL teams in the API to get the player names.

In [5]:
# create api request for teams
requests_teams = requests.get('https://statsapi.web.nhl.com/api/v1/teams')
teams = json.loads(requests_teams.text)['teams']

In [6]:
# current number of teams
len(teams)

32

In [7]:
# example of team entry in NHL API
teams[0]

{'id': 1,
 'name': 'New Jersey Devils',
 'link': '/api/v1/teams/1',
 'venue': {'name': 'Prudential Center',
  'link': '/api/v1/venues/null',
  'city': 'Newark',
  'timeZone': {'id': 'America/New_York', 'offset': -4, 'tz': 'EDT'}},
 'abbreviation': 'NJD',
 'teamName': 'Devils',
 'locationName': 'New Jersey',
 'firstYearOfPlay': '1982',
 'division': {'id': 18,
  'name': 'Metropolitan',
  'nameShort': 'Metro',
  'link': '/api/v1/divisions/18',
  'abbreviation': 'M'},
 'conference': {'id': 6, 'name': 'Eastern', 'link': '/api/v1/conferences/6'},
 'franchise': {'franchiseId': 23,
  'teamName': 'Devils',
  'link': '/api/v1/franchises/23'},
 'shortName': 'New Jersey',
 'officialSiteUrl': 'http://www.newjerseydevils.com/',
 'franchiseId': 23,
 'active': True}

In [8]:
# get list of teams and IDs to loop through API
team_ids = {t['name']:t['id'] for t in teams}

In [9]:
team_ids

{'New Jersey Devils': 1,
 'New York Islanders': 2,
 'New York Rangers': 3,
 'Philadelphia Flyers': 4,
 'Pittsburgh Penguins': 5,
 'Boston Bruins': 6,
 'Buffalo Sabres': 7,
 'Montréal Canadiens': 8,
 'Ottawa Senators': 9,
 'Toronto Maple Leafs': 10,
 'Carolina Hurricanes': 12,
 'Florida Panthers': 13,
 'Tampa Bay Lightning': 14,
 'Washington Capitals': 15,
 'Chicago Blackhawks': 16,
 'Detroit Red Wings': 17,
 'Nashville Predators': 18,
 'St. Louis Blues': 19,
 'Calgary Flames': 20,
 'Colorado Avalanche': 21,
 'Edmonton Oilers': 22,
 'Vancouver Canucks': 23,
 'Anaheim Ducks': 24,
 'Dallas Stars': 25,
 'Los Angeles Kings': 26,
 'San Jose Sharks': 28,
 'Columbus Blue Jackets': 29,
 'Minnesota Wild': 30,
 'Winnipeg Jets': 52,
 'Arizona Coyotes': 53,
 'Vegas Golden Knights': 54,
 'Seattle Kraken': 55}

In [10]:
# get roster for each team
dict_teamRosters = {}

for team in list(team_ids.keys()):
    team_id = team_ids[team]
    season = 2006
    list_rosters = []
    while (season < 2022):
        season_string = f'{season}{season+1}'
        request_roster_season = requests.get(f'https://statsapi.web.nhl.com/api/v1/teams/{team_id}/roster?expand=team.roster&season={season_string}')
        roster = json.loads(request_roster_season.text)
        df_roster = pd.DataFrame()
        try:
            for r in roster['roster']:
                player_id = r['person']['id']
                df_roster.loc[player_id,'Name'] = r['person']['fullName']
                df_roster.loc[player_id,'Position'] = r['position']['code']
            list_rosters.append(df_roster)
            season += 1
        except KeyError:
            season += 1
    df_team = pd.concat(list_rosters,axis=0)
    df_team = df_team.drop_duplicates()
    dict_teamRosters[team] = df_team

In [11]:
# example of a team roster
dict_teamRosters['New Jersey Devils']

,Name,Position
8456100,Jim Dowd,C
8458522,Richard Matvichuk,D
8458978,Sergei Brylin,L
8459454,Jay Pandolfo,L
8459457,Jamie Langenbrunner,R
...,...,...
8473541,Jonathan Bernier,G
8476903,Jon Gillies,G
8477202,Andrew Hammond,G
8481033,Akira Schmid,G


Since we are looking at all team rosters since 2006, we expect there to be some overlap because players get traded between teams or are signed to new contracts in free agency. Since there is a certain amount of data that will not change with a player's team or over time, such as height, birth date, nationality, etc., we will combine the rosters and remove any duplicates before using the API to obtain this data.

In [12]:
# combine roster dataframes into a single large dataframe for all players
df_players = pd.DataFrame()
for df_team in list(dict_teamRosters.values()):
    if df_players.empty:
        df_players = df_team
    else:
        df_players = pd.concat([df_players,df_team],axis=0)

In [13]:
df_players.head()

,Name,Position
8456100,Jim Dowd,C
8458522,Richard Matvichuk,D
8458978,Sergei Brylin,L
8459454,Jay Pandolfo,L
8459457,Jamie Langenbrunner,R


In [14]:
df_players.shape

(6802, 2)

In [15]:
# remove any duplicate players by ID
df_players['player_id'] = df_players.index
df_players = df_players.drop_duplicates(subset=['Name','player_id'])

In [16]:
df_players.shape

(3167, 3)

In [17]:
# remove player_id column
df_players = df_players.drop(columns=['player_id'],axis=1)

Now that we have a list of players on whom to find contracts, we can append relevant information from the API for every player. This data that we store here will be immutable, or applicable to the player over the course of their career, whereas other data, such as recent statistics, will change over the course of a player's career. We do, however, make the assumption that a player's height will not change during their career, which is reasonable because players must be 18 to play professionally in the NHL, and it is reasonable to conclude that a player is done growing by that age. We do not, however, make this assumption about weight.

In [18]:
# get immutable player data
for i in list(df_players.index):
    player_request = requests.get(f'https://statsapi.web.nhl.com/api/v1/people/{i}')
    player = json.loads(player_request.text)['people'][0]
    df_players.loc[i,'birthDate'] = player['birthDate']
    df_players.loc[i,'nationality'] = player['nationality']
    height_list = [int(player['height'].split(' ')[i].translate(str.maketrans('','',string.punctuation))) 
                   for i in [0,1]]
    df_players.loc[i,'height'] = height_list[0]*12+height_list[1]

In [19]:
df_players.head()

,Name,Position,birthDate,nationality,height
8456100,Jim Dowd,C,1968-12-25,USA,72.0
8458522,Richard Matvichuk,D,1973-02-05,CAN,75.0
8458978,Sergei Brylin,L,1974-01-13,RUS,70.0
8459454,Jay Pandolfo,L,1974-12-27,USA,73.0
8459457,Jamie Langenbrunner,R,1975-07-24,USA,73.0


In [20]:
# name index
df_players.index.name = 'Player ID'

Even though we removed duplicate player IDs before, we will double check for duplicates and missing values.

In [21]:
# check for missing data
df_players.isna().sum()

Name           0
Position       0
birthDate      0
nationality    0
height         0
dtype: int64

In [22]:
# check for duplicates - this is done by index (player ID)
df_players.duplicated().value_counts()

False    3167
dtype: int64

In [23]:
# confirm shape of dataframe
df_players.shape

(3167, 5)

In [24]:
# check for duplicate names
df_players.duplicated(subset='Name').sum()

9

Despite corresponding to different player IDs, there are several players who have identical names; while this is purely coincidental, when it comes to web-scraping, there will be some difficulty in determining what the player's corresponding URL should be, so for ease of implementation, those contracts will be disregarded.

In [25]:
# find duplicate names of players
bad_names = [name for name, count in Counter(df_players['Name']).items() if count != 1]

In [26]:
# drop players with same names from the dataframe
df_players = df_players[~df_players['Name'].isin(bad_names)]

In [27]:
# get new size of dataframe
df_players.shape

(3150, 5)

A player's draft record is also something that does not change over the course of the career - in fact it only happens once - so we will also populate the players dataframe with that information. For the rare case of undrafted players, there is some question of how they should be treated - without knowing if this is even a significant predictor, we will treat the missing values by filling with the median draft pick.

In [28]:
# get draft data
start_year = 1990
current_year = 2023

df_draft = pd.DataFrame()

while (start_year < current_year):
    draft_req = requests.get(f'https://statsapi.web.nhl.com/api/v1/draft/{start_year}')
    draft = json.loads(draft_req.text)['drafts'][0]
    for r in (draft['rounds']):
        for p in (r['picks']):
            player = p['prospect']['fullName']
            pick_overall = p['pickOverall']
            df_draft.loc[player,'Overall Pick'] = pick_overall
            df_draft.loc[player,'Year'] = start_year
    start_year += 1

In [29]:
df_draft.head()

,Overall Pick,Year
Mika Stromberg,211.0,1990.0
Tyler Ertel,212.0,1990.0
Brett Larson,213.0,1990.0
Tommy Soderstrom,214.0,1990.0
Michael Thompson,215.0,1990.0


In [30]:
df_draft.shape

(7804, 2)

In [31]:
# check for missing values
df_draft.isna().sum()

Overall Pick    0
Year            0
dtype: int64

In [32]:
# reset index to match draft data to player names
df_players = df_players.reset_index().set_index('Name')

In [33]:
df_players.head()

,Player ID,Position,birthDate,nationality,height
Name,,,,,
Jim Dowd,8456100,C,1968-12-25,USA,72.0
Richard Matvichuk,8458522,D,1973-02-05,CAN,75.0
Sergei Brylin,8458978,L,1974-01-13,RUS,70.0
Jay Pandolfo,8459454,L,1974-12-27,USA,73.0
Jamie Langenbrunner,8459457,R,1975-07-24,USA,73.0


In [34]:
# in case of any issues with casing in player names, switch all draft names to lowercase
df_draft_lower = df_draft.copy()
df_draft_lower.index = pd.Series(df_draft.index).apply(lambda x: x.lower())

for p in list(df_players.index):
    p_lower = p.lower()
    if p_lower in df_draft_lower.index:
        df_players.loc[p,'Overall Pick'] = df_draft_lower.loc[p_lower,'Overall Pick']
    else:
        df_players.loc[p,'Overall Pick'] = df_draft_lower['Overall Pick'].median()

In [35]:
df_players.head()

,Player ID,Position,birthDate,nationality,height,Overall Pick
Name,,,,,,
Jim Dowd,8456100,C,1968-12-25,USA,72.0,121.0
Richard Matvichuk,8458522,D,1973-02-05,CAN,75.0,8.0
Sergei Brylin,8458978,L,1974-01-13,RUS,70.0,42.0
Jay Pandolfo,8459454,L,1974-12-27,USA,73.0,32.0
Jamie Langenbrunner,8459457,R,1975-07-24,USA,73.0,35.0


In [36]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3150 entries, Jim Dowd to Matty Beniers
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Player ID     3150 non-null   int64  
 1   Position      3150 non-null   object 
 2   birthDate     3150 non-null   object 
 3   nationality   3150 non-null   object 
 4   height        3150 non-null   float64
 5   Overall Pick  3150 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 236.8+ KB


Since the date of birth feature is in string form, we need to convert this to a number that can be used further in the data-cleaning process.

In [37]:
df_players['birthYear'] = df_players['birthDate'].apply(lambda x: pd.to_numeric(x.split('-')[0]))

In [38]:
df_players.head()

,Player ID,Position,birthDate,nationality,height,Overall Pick,birthYear
Name,,,,,,,
Jim Dowd,8456100,C,1968-12-25,USA,72.0,121.0,1968
Richard Matvichuk,8458522,D,1973-02-05,CAN,75.0,8.0,1973
Sergei Brylin,8458978,L,1974-01-13,RUS,70.0,42.0,1974
Jay Pandolfo,8459454,L,1974-12-27,USA,73.0,32.0,1974
Jamie Langenbrunner,8459457,R,1975-07-24,USA,73.0,35.0,1975


In [39]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3150 entries, Jim Dowd to Matty Beniers
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Player ID     3150 non-null   int64  
 1   Position      3150 non-null   object 
 2   birthDate     3150 non-null   object 
 3   nationality   3150 non-null   object 
 4   height        3150 non-null   float64
 5   Overall Pick  3150 non-null   float64
 6   birthYear     3150 non-null   int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 261.4+ KB


Now that we have constructed a dataframe for players and some basic data, we need to obtain the salary cap data for each year to determine the contract value in terms of the salary cap.

In [40]:
# get salary cap data
url_cap = 'https://www.capfriendly.com/salary-cap'
response_cap = requests.get(url_cap)
tables_cap = pd.read_html(response_cap.text)[0]
tables_cap['SEASON'] = tables_cap['SEASON'].apply(lambda x: int(x.split('-')[0]))
cap_table = tables_cap.set_index('SEASON').drop(columns=['CONFIRMED','% CHANGE']).applymap(lambda x: 
            int(x.replace('$','').replace(',','')))

In [41]:
cap_table

,UPPER LIMIT,LOWER LIMIT,MIN. SALARY
SEASON,,,
2025,92000000,68000000,775000
2024,87500000,64700000,775000
2023,83500000,61700000,775000
2022,82500000,61000000,750000
2021,81500000,60200000,750000
2020,81500000,60200000,700000
2019,81500000,60200000,700000
2018,79500000,58800000,650000
2017,75000000,55400000,650000


Now we can use defined functions to combine and clean data that will be used to make the dataframe on which the models can be run. To do this, we need to split the players into goalies and non-goalies - this is because goalie metrics are an entirely diffrent set of statistics.

In [42]:
# split the player dataframe into goalie and non-goalie players
df_players_g = df_players[df_players['Position'] == 'G']
df_players_ng = df_players[df_players['Position'] != 'G']

In [43]:
# confirm that only goalies appear in goalie set
df_players_g['Position'].unique()

array(['G'], dtype=object)

In [44]:
# confirm that no goalies appear in non-goalie set
df_players_ng['Position'].unique()

array(['C', 'D', 'L', 'R'], dtype=object)

In [45]:
# get player contract data for goalies
dict_contracts_g = getPlayerContractData(df_players_g,cap_table)

In [46]:
# construct dataframe with statistics for goalies
dataTable_g = getDataTable_g(dict_contracts_g)

In [47]:
dataTable_g.head()

,Value,Length,AAV,Contract Age,Cap Hit,Position,Nationality,Height,Overall Pick,GAMES_PLAYED_t,...,Sv%_avg,GP_PLAYOFF_avg,GAA_PLAYOFF_avg,Sv%_PLAYOFF_avg,GA60_avg,xGA60_avg,GSAx60_avg,GA60_PLAYOFF_avg,xGA60_PLAYOFF_avg,GSAx60_PLAYOFF_avg
2006,31200000.0,6.0,5200000.0,34.0,0.118182,G,CAN,74.0,20.0,73.0,...,0.914000,7.000000,2.435000,0.912500,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2012,9000000.0,2.0,4500000.0,40.0,0.075000,G,CAN,74.0,20.0,59.0,...,0.909000,9.666667,1.710000,0.599333,2.170000,1.896667,-0.28,1.123333,1.276667,0.156667
2014,700000.0,1.0,700000.0,42.0,0.010145,G,CAN,74.0,20.0,39.0,...,0.903333,8.000000,0.706667,0.305667,2.123333,1.766667,-0.36,0.513333,0.603333,0.093333
2006,450000.0,1.0,450000.0,29.0,0.010227,G,USA,74.0,215.0,13.0,...,0.916500,0.500000,0.000000,0.500000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2007,500000.0,1.0,500000.0,30.0,0.009940,G,USA,74.0,215.0,6.0,...,0.885000,0.500000,0.000000,0.500000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


In [48]:
# get player contract data for non-goalies
dict_contracts_ng = getPlayerContractData(df_players_ng,cap_table)

In [49]:
# construct dataframe with statistics for non-goalies
dataTable_ng = getDataTable_ng(dict_contracts_ng)

In [50]:
dataTable_ng.shape

(5665, 125)

In [51]:
dataTable_ng.head()

,Value,Length,AAV,Contract Age,Cap Hit,Position,Nationality,Height,Overall Pick,GAMES_PLAYED_t,...,ixG_PLAYOFF_avg,xG±/60_PLAYOFF_avg,RelxG±/60_PLAYOFF_avg,C±/60_PLAYOFF_avg,RelC±/60_PLAYOFF_avg,G_PG_avg,P_PG_avg,PIM_PG_avg,G_PG_PLAYOFF_avg,P_PG_PLAYOFF_avg
2008,7500000.0,3.0,2500000.0,34.0,0.044092,L,USA,73.0,32.0,54.0,...,0.203333,-0.143333,-0.093333,-3.380000,-1.830000,0.167570,0.339205,0.233363,0.067340,0.215488
2011,600000.0,1.0,600000.0,37.0,0.009331,L,USA,73.0,32.0,0.0,...,0.310000,-0.080000,-0.055000,-3.365000,0.485000,0.079445,0.168506,0.139660,0.071429,0.071429
2012,600000.0,1.0,600000.0,38.0,0.010000,L,USA,73.0,32.0,62.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.046526,0.110732,0.122208,0.000000,0.000000
2010,833334.0,1.0,833334.0,36.0,0.014029,L,USA,73.0,32.0,52.0,...,0.410000,-0.196667,-0.130000,-5.623333,-1.506667,0.127038,0.260485,0.228909,0.047619,0.047619
2005,1655130.0,1.0,1655130.0,30.0,0.042439,R,USA,73.0,35.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.235365,0.597847,0.822327,0.229167,0.575000


In [52]:
# export data to csv for use in modeling notebook
dataTable_ng.to_csv('data/data_ng.csv',index=False)

Now that we have the data we are going to use for modelling, we can similarly generate the same data for unrestricted free agents for whom we want to project contract values. First, we need to get a list of unrestricted free agents.

In [53]:
# get list of new UFAs
url_ufa = 'https://www.spotrac.com/nhl/free-agents/ufa/'
response_ufa = requests.get(url_ufa)
tables_ufa = pd.read_html(response_ufa.text)[0]

In [54]:
tables_ufa

,Player (259),Team,Type,Age,Pos.,Height,Weight,Shoots,GP,G,A,P,W,GAA,SV%,2022-2023 Cap Hit
0,Patrick Kane,NYR,UFA,34.6,RW,"5' 10""",177.0,L,73.0,21.0,36.0,57.0,NaN,NaN,NaN,"$10,500,000"
1,Jonathan Toews,CHI,UFA,35.1,C,"6' 2""",201.0,L,53.0,15.0,16.0,31.0,NaN,NaN,NaN,"$10,500,000"
2,Oliver Ekman-Larsson,VAN,UFA,31.9,D,"6' 2""",200.0,L,54.0,2.0,20.0,22.0,NaN,NaN,NaN,"$8,250,000"
3,Ryan O'Reilly,TOR,UFA,32.3,C,"6' 1""",207.0,L,53.0,16.0,14.0,30.0,NaN,NaN,NaN,"$7,500,000"
4,Vladimir Tarasenko,NYR,UFA,31.5,RW,"6' 1""",228.0,L,69.0,18.0,32.0,50.0,NaN,NaN,NaN,"$7,500,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Michael Hutchinson,CBJ,UFA,33.2,G,"6' 3""",201.0,R,16.0,NaN,1.0,1.0,2.0,4.29,0.875,"$750,000"
255,Jon Gillies,CBJ,UFA,29.3,G,"6' 6""",223.0,L,3.0,NaN,NaN,NaN,1.0,4.58,0.864,"$750,000"
256,Charles Hudon,COL,UFA,28.9,LW,"5' 10""",190.0,L,9.0,NaN,NaN,NaN,NaN,NaN,NaN,"$750,000"
257,Max Willman,PHI,UFA,28.3,C,"6' 0""",184.0,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$750,000"


In [55]:
list_players = list(tables_ufa.iloc[:,0])
current_salaries = list(tables_ufa.iloc[:,-1])

In [56]:
len(list_players)

259

In [57]:
list_players[0:10]

['Patrick Kane',
 'Jonathan Toews',
 'Oliver Ekman-Larsson',
 "Ryan O'Reilly",
 'Vladimir Tarasenko',
 'James Van Riemsdyk',
 'John Klingberg',
 'Max Pacioretty',
 'Milan Lucic',
 'Mathew Dumba']

In [58]:
# get new dataframe for UFAs
df_players_new = df_players_ng[df_players_ng.index.isin(list_players)]

In [59]:
df_players_new.head()

,Player ID,Position,birthDate,nationality,height,Overall Pick,birthYear
Name,,,,,,,
Zach Parise,8470610,L,1984-07-28,USA,71.0,17.0,1984
John Moore,8475186,D,1990-11-19,USA,75.0,21.0,1990
Miles Wood,8477425,L,1995-09-13,USA,74.0,100.0,1995
Steven Santini,8477463,D,1995-03-07,USA,74.0,42.0,1995
Will Butcher,8477355,D,1995-01-06,USA,70.0,123.0,1995


In [60]:
df_players_new.shape

(194, 7)

In [69]:
# create a new dataframe with names and current salaries to merge later
df_temp = pd.DataFrame({'Current Salary': current_salaries},index=list_players)

In [70]:
df_temp.head()

,Current Salary
Patrick Kane,"$10,500,000"
Jonathan Toews,"$10,500,000"
Oliver Ekman-Larsson,"$8,250,000"
Ryan O'Reilly,"$7,500,000"
Vladimir Tarasenko,"$7,500,000"


Now that we have the dataframe of players and their relevant information, we can re-configure a defined function to get a matching table for each player - using the contract year 2023 to get the relevant statistics.

In [71]:
def getPlayerContractData_newContracts(df_players,contract_year):
    """
    Returns a dictionary with key corresponding to player and value corresponding to dictionary of two tables:
    The first table is contract specific data and player-immutable data, and the second table is advanced player
    statistics broken down by year
    """
    dict_contracts = {}
    
    for p in list(df_players.index):
        first,last = p.split(' ')[0],p.split(' ')[1]
        url = f'https://www.capfriendly.com/players/{first}-{last}'
        response = requests.get(url)
        try:
            tables = pd.read_html(response.text)
        except ValueError:
            continue
        
        if len(tables) < 2:
            continue
        dict_player = {}
        df_newContract = pd.DataFrame()
        
        y = contract_year
        df_newContract.loc[y,'Name'] = p
        df_newContract.loc[y,'ContractAge'] = y-df_players.loc[p,'birthYear']
        
        df_newContract['Position'] = df_players.loc[p,'Position']
        df_newContract['Nationality'] = df_players.loc[p,'nationality']
        df_newContract['Height'] = df_players.loc[p,'height']
        df_newContract['Overall Pick'] = df_players.loc[p,'Overall Pick']
        
        summary_table = tables[-1]
        summary_table = summary_table[~summary_table['SEASON'].isna()].iloc[:-1,:]
        summary_table = summary_table[summary_table['LEAGUE'] == 'NHL']
        summary_table['SEASON ENDING'] = (summary_table['SEASON'].apply(lambda x: int(x.split('-')[0][:2] + 
                                                                                      x.split('-')[1])))
        summary_table = summary_table.set_index('SEASON ENDING')

        dict_player['Summary'] = df_newContract
        dict_player['Stats'] = summary_table
        dict_contracts[p] = dict_player
        
    return dict_contracts

In [72]:
# get player data for new contract players
dict_new_contracts = getPlayerContractData_newContracts(df_players_new,2023)

In [83]:
# construct dataframe with statistics for new contract players
dataTable_ng_new = getDataTable_ng(dict_new_contracts)

In [84]:
dataTable_ng_new.head()

,Name,ContractAge,Position,Nationality,Height,Overall Pick,GAMES_PLAYED_t,G_t,A_t,P_t,...,ixG_PLAYOFF_avg,xG±/60_PLAYOFF_avg,RelxG±/60_PLAYOFF_avg,C±/60_PLAYOFF_avg,RelC±/60_PLAYOFF_avg,G_PG_avg,P_PG_avg,PIM_PG_avg,G_PG_PLAYOFF_avg,P_PG_PLAYOFF_avg
2023,Zach Parise,39.0,L,USA,71.0,17.0,82.0,21.0,13.0,34.0,...,0.563333,-0.200000,0.073333,-14.293333,-6.610000,0.198193,0.413821,0.255827,0.166667,0.250000
2023,John Moore,33.0,D,USA,75.0,21.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.271429,0.485714,0.000000,0.000000
2023,Miles Wood,28.0,L,USA,74.0,100.0,76.0,13.0,14.0,27.0,...,0.310000,-0.043333,-0.086667,-6.646667,-5.193333,0.160048,0.269936,0.953535,0.083333,0.083333
2023,Steven Santini,28.0,D,USA,74.0,42.0,4.0,0.0,1.0,1.0,...,0.050000,0.630000,1.353333,29.753333,37.203333,0.000000,0.194444,0.166667,0.000000,0.000000
2023,Will Butcher,28.0,D,USA,70.0,123.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.048766,0.347239,0.043478,0.000000,0.000000


In [85]:
dataTable_ng_new.shape

(178, 122)

In [86]:
# now we will append the salary data for the given players
dataTable_ng_new = dataTable_ng_new.set_index('Name')
data_new = dataTable_ng_new.merge(df_temp,left_index=True,right_index=True)

In [87]:
data_new.head()

,ContractAge,Position,Nationality,Height,Overall Pick,GAMES_PLAYED_t,G_t,A_t,P_t,PLUS_MINUS_t,...,xG±/60_PLAYOFF_avg,RelxG±/60_PLAYOFF_avg,C±/60_PLAYOFF_avg,RelC±/60_PLAYOFF_avg,G_PG_avg,P_PG_avg,PIM_PG_avg,G_PG_PLAYOFF_avg,P_PG_PLAYOFF_avg,Current Salary
Zach Parise,39.0,L,USA,71.0,17.0,82.0,21.0,13.0,34.0,7.0,...,-0.200000,0.073333,-14.293333,-6.610000,0.198193,0.413821,0.255827,0.166667,0.250000,"$6,371,794"
John Moore,33.0,D,USA,75.0,21.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.271429,0.485714,0.000000,0.000000,"$2,750,000"
Miles Wood,28.0,L,USA,74.0,100.0,76.0,13.0,14.0,27.0,1.0,...,-0.043333,-0.086667,-6.646667,-5.193333,0.160048,0.269936,0.953535,0.083333,0.083333,"$3,200,000"
Steven Santini,28.0,D,USA,74.0,42.0,4.0,0.0,1.0,1.0,0.0,...,0.630000,1.353333,29.753333,37.203333,0.000000,0.194444,0.166667,0.000000,0.000000,"$750,000"
Will Butcher,28.0,D,USA,70.0,123.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.048766,0.347239,0.043478,0.000000,0.000000,"$750,000"


In [88]:
# reset the index so 'Name' is its own column
data_new = data_new.reset_index()

In [89]:
data_new.head()

,index,ContractAge,Position,Nationality,Height,Overall Pick,GAMES_PLAYED_t,G_t,A_t,P_t,...,xG±/60_PLAYOFF_avg,RelxG±/60_PLAYOFF_avg,C±/60_PLAYOFF_avg,RelC±/60_PLAYOFF_avg,G_PG_avg,P_PG_avg,PIM_PG_avg,G_PG_PLAYOFF_avg,P_PG_PLAYOFF_avg,Current Salary
0,Zach Parise,39.0,L,USA,71.0,17.0,82.0,21.0,13.0,34.0,...,-0.200000,0.073333,-14.293333,-6.610000,0.198193,0.413821,0.255827,0.166667,0.250000,"$6,371,794"
1,John Moore,33.0,D,USA,75.0,21.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.271429,0.485714,0.000000,0.000000,"$2,750,000"
2,Miles Wood,28.0,L,USA,74.0,100.0,76.0,13.0,14.0,27.0,...,-0.043333,-0.086667,-6.646667,-5.193333,0.160048,0.269936,0.953535,0.083333,0.083333,"$3,200,000"
3,Steven Santini,28.0,D,USA,74.0,42.0,4.0,0.0,1.0,1.0,...,0.630000,1.353333,29.753333,37.203333,0.000000,0.194444,0.166667,0.000000,0.000000,"$750,000"
4,Will Butcher,28.0,D,USA,70.0,123.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.048766,0.347239,0.043478,0.000000,0.000000,"$750,000"


It is important to note that of the 267 free agents we first scraped the names of, we only recover data for 178 of them - this is because of the free agents that are technically available, it is likely that some of them have not played in the last three years or have been playing in the minor league for that period while technically on a major team contract. Since we do not have adequate information for those players, they are excluded.

In [90]:
# save new data
data_new.to_csv('data/new_contract_players.csv',index=False)